# Codebook:

 - This notebook contains all the code used to run the analysis. 
 - In case you want to test the code using your data, all you need to do is to change `dir_bas` path. 

In [ ]:
# Importing the libraries:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import sqlite3 
import numpy as np 
import glob
import nibabel as ni 
from nibabel import nifti1 
import ia870 as MM 
import dtimp as dtimp 
import functions_will as FW
import ia636 as ia
import sys

In [ ]:
# Defining the functions we will use to segment and parcelate:
def run_analysis(rootdir):  

    eigvals, eigvects, T3 = dtimp.loadNiftiDTI(rootdir, reorient=True)

    FA,MD = dtimp.getFractionalAnisotropy(eigvals)
    FA[np.isnan(FA)] = 0
    FA[FA>1] = 1

    fissure, FA_mean = dtimp.getFissureSlice(eigvals, FA)

    wFA = FA*abs(eigvects[0,0]) #weighted FA
    
    return wFA, FA, MD, fissure, eigvals, eigvects

def corpusCallosumParcellationGeometric (segmentation, scheme = 'HOFER'):
    
    SCHEMES = ['HOFER', 'WITELSON']
    scheme = scheme.upper()
    if not scheme in SCHEMES:
        raise Exception('Unknown scheme!')

    def coef_linear (a, p):
        return p[0]-a*p[1] 

    def predicty(x, a, b):
        return a*x + b

    def predictx(y, a, b):
        return (y-b)/a

    # Base and normal vectors 
    M,N = np.nonzero(segmentation)
    minN = np.min(N)
    maxN = np.max(N)
    minM = segmentation[:,minN].nonzero()[0].mean()
    maxM = segmentation[:,maxN].nonzero()[0].mean()
    p1 = np.array([minM, minN])
    p2 = np.array([maxM, maxN])

    base_v = p2 - p1
    base_length = np.sqrt((base_v**2).sum())
    base_v = base_v / np.sqrt((base_v**2).sum())
    cut_v = np.array([-base_v[1], base_v[0]])

    # Line's coefficients
    hofer = np.array([1.0/6, 1.0/2, 2.0/3, 3.0/4]).reshape(4,1)
    witelson = np.array([1.0/3, 1.0/2, 2.0/3, 4.0/5]).reshape(4,1)

    if scheme == 'HOFER':
        P = p1 + hofer*base_length*base_v

    if scheme == 'WITELSON':
        P = p1 + witelson*base_length*base_v

    p3, p4, p5, p6 = P

    rbase_A = base_v[0]/base_v[1]
    rbase_B = p1[0]-rbase_A*p1[1]
    rA = cut_v[0]/cut_v[1]
    r3B = coef_linear(rA, p3)
    r4B = coef_linear(rA, p4)
    r5B = coef_linear(rA, p5)
    r6B = coef_linear(rA, p6)

    # Rotulating the mask    
    H,W = np.shape(segmentation)
    Parcellation = np.zeros((H,W), dtype='int')

    y,x = segmentation.nonzero()
    labels = np.zeros(y.size, dtype='int')
    above_base = y <= predicty(x, rbase_A, rbase_B)
    left_r3 = x <= predictx(y, rA, r3B)
    left_r4 = x <= predictx(y, rA, r4B)
    left_r5 = x <= predictx(y, rA, r5B)
    left_r6 = x <= predictx(y, rA, r6B)

    labels[np.logical_and(left_r3==False, left_r4)] = 2
    labels[np.logical_and(left_r4==False, left_r5)] = 3
    labels[np.logical_and(left_r5==False, left_r6)] = 4
    labels[np.logical_or(np.logical_and(above_base==False, left_r4), left_r3)] = 1
    labels[np.logical_or(np.logical_and(above_base==False, left_r5==False), left_r6==False)] = 5

    Parcellation[segmentation] = labels
    return Parcellation 

## Controls:

 - Here is the analysis for controls.

In [ ]:
# Defining the path: 
dir_bas = "data"
dirs = glob.glob(dir_bas+"/*C*")

acumC = np.zeros((3,6,4))
acumC2 = np.zeros((3,6,4))
seedx = [31,28,25]
seedy = [152,101,125]
indx = 0 

for candidate in dirs: 
    wFA, FA, MD, fissure, eigvals, eigvects = run_analysis(candidate)
    escala = [3*FA[fissure,:,:].shape[-2],2*FA[fissure,:,:].shape[-1]]
    wFA_ms = wFA[fissure,:,:]
    eigvects_ms = abs(eigvects[0,:,fissure]) 

    wFA_ss = np.load(candidate+'/segmentacao/wFA_ss.npy').astype(np.bool)
    escala = [wFA_ss.shape[-2],wFA_ss.shape[-1]]
    teste = mpimg.imread(candidate+'/mask_AF.png')[:,:,0]
    mask_p3 = np.array(teste).astype(np.bool)
    mask_sem = FW.resizedti(mask_p3,escala).astype(bool)
    con_mask_cc = np.logical_xor(MM.iaero(mask_sem),mask_sem)
    
    # ROQS code:
    seg_mask_seed_ero = MM.iaero(mask_sem)
    seed = [seedx[indx],seedy[indx]]

    max_comp_in = np.argmax(eigvects_ms[:,seed[0],seed[1]],axis=0)
    max_comp_in = np.argmax(ia.iahistogram(max_comp_in))
    Cmax_seed = eigvects_ms[max_comp_in,seed[0],seed[1]]

    princ = np.argmax(eigvects_ms,axis=0)
    fsc = princ == max_comp_in

    alpha = 0.3
    beta = 0.3
    gamma = 0.5
    FA_ms = FA[fissure]
    MA = (FA_ms-np.amax(FA_ms)*alpha)/(np.amax(FA_ms)*beta)+gamma
    ssc = np.clip(np.amax(eigvects_ms*MA,axis=0),0,1)
    ssc = ssc*fsc
    mask_cc = ssc > Cmax_seed-0.1
    fr = MM.ialabel(mask_cc,MM.iasebox())
    fra = MM.iablob(fr,'area')
    mask_cc = fra == np.unique(fra)[-1]
    np.save(candidate+'/segmentacao/mask_ROQS', mask_cc)
    segmentation = mask_cc # Saving the segmentation mask 

    # Parcelattion code:
    Ts = np.array([[4.0,0,0],[0,4,0],[0,0,1]])
    t = 0.2
    wFA_slice = wFA[fissure]
    FA_slice = FA[fissure]
    MD_slice = MD[fissure]
    fc = 8.0
    Ts = np.array([[fc,0,0],[0,fc,0],[0,0,1]])

    #Hofer and Frahm code:
    scheme = 'HOFER'
    geometricw = corpusCallosumParcellationGeometric(segmentation, scheme)

    # If you would like to see the segmentation and parcellation images from each subject:
    
    plt.figure(figsize=(10,10))
    plt.imshow(ia.ianormalize(segmentation))
    plt.title('Segmentation')
    plt.figure(2)
    plt.figure(figsize=(10,10))
    plt.imshow(ia.ianormalize(geometricw))
    plt.title('Hofer and Frahm Parcellation')
    
    hofer = geometricw

    labels = geometricw.max(axis=0)
    labels = labels[labels[:-1]!=labels[1:]][1:].copy()    
    measuresw = []
    measuresw.append([FA_slice[segmentation].mean(), FA_slice[segmentation].std(), MD_slice[segmentation].mean(), MD_slice[segmentation].std()])
    for l in labels:
        selection = (geometricw==l)
        measuresw.append([FA_slice[selection].mean(), FA_slice[selection].std(), MD_slice[selection].mean(), MD_slice[selection].std()])
    measuresw = np.array(measuresw)
    acumC[indx] = measuresw
    

    #Witelson code:
    scheme = 'WITELSON'
    geometricw = corpusCallosumParcellationGeometric(segmentation, scheme)
    
    # If you would like to see the segmentation and parcellation images, discomment the lines below:
    
    plt.figure(3)
    plt.figure(figsize=(10,10))
    plt.imshow(ia.ianormalize(geometricw))
    plt.title('Witelson Parcellation')
    
    witelson = geometricw
    
    labels = geometricw.max(axis=0)
    labels = labels[labels[:-1]!=labels[1:]][1:].copy()    
    measuresw = []
    measuresw.append([FA_slice[segmentation].mean(), FA_slice[segmentation].std(), MD_slice[segmentation].mean(), MD_slice[segmentation].std()])
    for l in labels:
        selection = (geometricw==l)
        measuresw.append([FA_slice[selection].mean(), FA_slice[selection].std(), MD_slice[selection].mean(), MD_slice[selection].std()])
    measuresw = np.array(measuresw)

    acumC2[indx] = measuresw
    indx += 1 

In [ ]:
# Calculating the average for all controls:
auxcumC = np.average(acumC,axis=0)
auxcumC2 = np.average(acumC2,axis=0)

print '====================================================='
print '           Hofer and Frahm Parcellation '
print '====================================================='
print '           FA (mean)   FA (std)  MD (mean)  MD (std)'
print('Corpus : %.3e, %.3e, %.3e, %.3e' %(auxcumC[0,0], auxcumC[0,1],auxcumC[0,2],auxcumC[0,3]))
print('Slice 1: %.3e, %.3e, %.3e, %.3e' %(auxcumC[1,0], auxcumC[1,1],auxcumC[1,2],auxcumC[1,3]))
print('Slice 2: %.3e, %.3e, %.3e, %.3e' %(auxcumC[2,0], auxcumC[2,1],auxcumC[2,2],auxcumC[2,3]))
print('Slice 3: %.3e, %.3e, %.3e, %.3e' %(auxcumC[3,0], auxcumC[3,1],auxcumC[3,2],auxcumC[3,3]))
print('Slice 4: %.3e, %.3e, %.3e, %.3e' %(auxcumC[4,0], auxcumC[4,1],auxcumC[4,2],auxcumC[4,3]))
print('Slice 5: %.3e, %.3e, %.3e, %.3e' %(auxcumC[5,0], auxcumC[4,1],auxcumC[5,2],auxcumC[5,3]))
print('\n\n')
print '====================================================='
print '           Witelson Parcellation '
print '====================================================='
print '           FA (mean)   FA (std)  MD (mean)  MD (std)'
print('Corpus : %.3e, %.3e, %.3e, %.3e' %(auxcumC2[0,0], auxcumC2[0,1],auxcumC2[0,2],auxcumC2[0,3]))
print('Slice 1: %.3e, %.3e, %.3e, %.3e' %(auxcumC2[1,0], auxcumC2[1,1],auxcumC2[1,2],auxcumC2[1,3]))
print('Slice 2: %.3e, %.3e, %.3e, %.3e' %(auxcumC2[2,0], auxcumC2[2,1],auxcumC2[2,2],auxcumC2[2,3]))
print('Slice 3: %.3e, %.3e, %.3e, %.3e' %(auxcumC2[3,0], auxcumC2[3,1],auxcumC2[3,2],auxcumC2[3,3]))
print('Slice 4: %.3e, %.3e, %.3e, %.3e' %(auxcumC2[4,0], auxcumC2[4,1],auxcumC2[4,2],auxcumC2[4,3]))
print('Slice 5: %.3e, %.3e, %.3e, %.3e' %(auxcumC2[5,0], auxcumC2[4,1],auxcumC2[5,2],auxcumC2[5,3]))


## Patients:

 - Here is the analysis for patients.

In [ ]:
# Defining the path: 
dir_bas = "data"
dirs = glob.glob(dir_bas+"/*P*")

acumP = np.zeros((6,6,4))
acumP2 = np.zeros((6,6,4))
seedx = [25,25,30,29,29,29]
seedy = [132,111,95,146,106,67]
indx = 0 

for candidate in dirs: 
    wFA, FA, MD, fissure, eigvals, eigvects = run_analysis(candidate)
    escala = [3*FA[fissure,:,:].shape[-2],2*FA[fissure,:,:].shape[-1]]
    wFA_ms = wFA[fissure,:,:]
    eigvects_ms = abs(eigvects[0,:,fissure]) 

    wFA_ss = np.load(candidate+'/segmentacao/wFA_ss.npy').astype(np.bool)
    escala = [wFA_ss.shape[-2],wFA_ss.shape[-1]]
    teste = mpimg.imread(candidate+'/mask_AF.png')[:,:,0]
    mask_p3 = np.array(teste).astype(np.bool)
    mask_sem = FW.resizedti(mask_p3,escala).astype(bool)
    con_mask_cc = np.logical_xor(MM.iaero(mask_sem),mask_sem)
    
    # ROQS code:
    seg_mask_seed_ero = MM.iaero(mask_sem)
    seed = [seedx[indx],seedy[indx]]

    max_comp_in = np.argmax(eigvects_ms[:,seed[0],seed[1]],axis=0)
    max_comp_in = np.argmax(ia.iahistogram(max_comp_in))
    Cmax_seed = eigvects_ms[max_comp_in,seed[0],seed[1]]

    princ = np.argmax(eigvects_ms,axis=0)
    fsc = princ == max_comp_in

    alpha = 0.3
    beta = 0.3
    gamma = 0.5
    FA_ms = FA[fissure]
    MA = (FA_ms-np.amax(FA_ms)*alpha)/(np.amax(FA_ms)*beta)+gamma
    ssc = np.clip(np.amax(eigvects_ms*MA,axis=0),0,1)
    ssc = ssc*fsc
    mask_cc = ssc > Cmax_seed-0.1
    fr = MM.ialabel(mask_cc,MM.iasebox())
    fra = MM.iablob(fr,'area')
    mask_cc = fra == np.unique(fra)[-1]
    np.save(candidate+'/segmentacao/mask_ROQS', mask_cc)
    segmentation = mask_cc # Saving the segmentation mask 

    # Parcelattion code:
    Ts = np.array([[4.0,0,0],[0,4,0],[0,0,1]])
    t = 0.2
    wFA_slice = wFA[fissure]
    FA_slice = FA[fissure]
    MD_slice = MD[fissure]
    fc = 8.0
    Ts = np.array([[fc,0,0],[0,fc,0],[0,0,1]])
    
    #Hofer and Frahm code:
    scheme = 'HOFER'
    geometricw = corpusCallosumParcellationGeometric(segmentation, scheme)

    # If you would like to see the segmentation and parcellation images from each subject:
    
    plt.figure(figsize=(10,10))
    plt.imshow(ia.ianormalize(segmentation))
    plt.title('Segmentation')
    plt.figure(2)
    plt.figure(figsize=(10,10))
    plt.imshow(ia.ianormalize(geometricw))
    plt.title('Hofer and Frahm Parcellation')

    labels = geometricw.max(axis=0)
    labels = labels[labels[:-1]!=labels[1:]][1:].copy()    
    measuresw = []
    measuresw.append([FA_slice[segmentation].mean(), FA_slice[segmentation].std(), MD_slice[segmentation].mean(), MD_slice[segmentation].std()])
    for l in labels:
        selection = (geometricw==l)
        measuresw.append([FA_slice[selection].mean(), FA_slice[selection].std(), MD_slice[selection].mean(), MD_slice[selection].std()])
    measuresw = np.array(measuresw)
    acumP[indx] = measuresw
    

    #Witelson code:
    scheme = 'WITELSON'
    geometricw = corpusCallosumParcellationGeometric(segmentation, scheme)

    # If you would like to see the segmentation and parcellation images from each subject:
    
    plt.figure(3)
    plt.figure(figsize=(10,10))
    plt.imshow(ia.ianormalize(geometricw))
    plt.title('Witelson Parcellation')

    labels = geometricw.max(axis=0)
    labels = labels[labels[:-1]!=labels[1:]][1:].copy()    
    measuresw = []
    measuresw.append([FA_slice[segmentation].mean(), FA_slice[segmentation].std(), MD_slice[segmentation].mean(), MD_slice[segmentation].std()])
    for l in labels:
        selection = (geometricw==l)
        measuresw.append([FA_slice[selection].mean(), FA_slice[selection].std(), MD_slice[selection].mean(), MD_slice[selection].std()])
    measuresw = np.array(measuresw)
    
    acumP2[indx] = measuresw
    indx += 1

In [ ]:
# Calculating the average for all patients:
auxcumP = np.average(acumP,axis=0)
auxcumP2 = np.average(acumP2,axis=0)

print '====================================================='
print '           Hofer and Frahm Parcellation '
print '====================================================='
print '           FA (mean)   FA (std)  MD (mean)  MD (std)'
print('Corpus : %.3e, %.3e, %.3e, %.3e' %(auxcumP[0,0], auxcumP[0,1],auxcumP[0,2],auxcumP[0,3]))
print('Slice 1: %.3e, %.3e, %.3e, %.3e' %(auxcumP[1,0], auxcumP[1,1],auxcumP[1,2],auxcumP[1,3]))
print('Slice 2: %.3e, %.3e, %.3e, %.3e' %(auxcumP[2,0], auxcumP[2,1],auxcumP[2,2],auxcumP[2,3]))
print('Slice 3: %.3e, %.3e, %.3e, %.3e' %(auxcumP[3,0], auxcumP[3,1],auxcumP[3,2],auxcumP[3,3]))
print('Slice 4: %.3e, %.3e, %.3e, %.3e' %(auxcumP[4,0], auxcumP[4,1],auxcumP[4,2],auxcumP[4,3]))
print('Slice 5: %.3e, %.3e, %.3e, %.3e' %(auxcumP[5,0], auxcumP[4,1],auxcumP[5,2],auxcumP[5,3]))
print('\n\n')
print '====================================================='
print '           Witelson Parcellation '
print '====================================================='
print '           FA (mean)   FA (std)  MD (mean)  MD (std)'
print('Corpus : %.3e, %.3e, %.3e, %.3e' %(auxcumP2[0,0], auxcumP2[0,1],auxcumP2[0,2],auxcumP2[0,3]))
print('Slice 1: %.3e, %.3e, %.3e, %.3e' %(auxcumP2[1,0], auxcumP2[1,1],auxcumP2[1,2],auxcumP2[1,3]))
print('Slice 2: %.3e, %.3e, %.3e, %.3e' %(auxcumP2[2,0], auxcumP2[2,1],auxcumP2[2,2],auxcumP2[2,3]))
print('Slice 3: %.3e, %.3e, %.3e, %.3e' %(auxcumP2[3,0], auxcumP2[3,1],auxcumP2[3,2],auxcumP2[3,3]))
print('Slice 4: %.3e, %.3e, %.3e, %.3e' %(auxcumP2[4,0], auxcumP2[4,1],auxcumP2[4,2],auxcumP2[4,3]))
print('Slice 5: %.3e, %.3e, %.3e, %.3e' %(auxcumP2[5,0], auxcumP2[4,1],auxcumP2[5,2],auxcumP2[5,3]))
